# Classification of TL2LA: Pattern-based Contribution Method
<br> ------------------------------------------------------------------------
<br> Experiment with the Pattern-based Contribution Method 
<br> and apply it to signle scenes of the transformed Lyft
<br> Level 5 dataset
<br>
<br> tl2la
<br> Copyright (c) 2023 Andreas Weber. All Rights Reserved.
<br> ------------------------------------------------------------------------


In [1]:
from src.assignment.assignmet_pattern import (fetch_contribution, detect_pattern, heristic_prediction_for_scenes, heristic_prediction_for_signle_scene, apply_frame_contribution, generalize_over_frames, generalize_over_scenes_by_class_majority)
from src.common.helper import (get_pickled_dataset_index, init_dataset)
from src.common.definitions import DATASET_TYPE, TL_STATUS, TURN
from src.dataset_operations.extended_map_api import ExtendedMapAPI
from src.dataset_operations.junction_dataset import JunctionDataset

##  Initialisation of required Objects

In [2]:
dataset_type = "trainfull"
cfg, dm, zarr_dataset = init_dataset(config_name=dataset_type)
mapAPI = ExtendedMapAPI.from_config(dm, cfg)
junctionDataset = JunctionDataset(cfg, mapAPI, zarr_dataset)
extended_branch_df = junctionDataset.get_branch_dataset(DATASET_TYPE.EXTENDED)

## Motion Pattern Detection

In [6]:
# 1.) Moving Scenario
agent_pattern = detect_pattern(dist=2.0, velocity=10.0, acceleration=0.0)
print(f"Detected Pattern: {agent_pattern.name}")

# 2.) Deceleration Scenario
agent_pattern = detect_pattern(dist=2.0, velocity=10.0, acceleration=-3.0)
print(f"Detected Pattern: {agent_pattern.name}")

# 3.) Acceleration Scenario
agent_pattern = detect_pattern(dist=1.0, velocity=0.1, acceleration=3.0)
print(f"Detected Pattern: {agent_pattern.name}")


Detected Pattern: MOVING
Detected Pattern: DECELERATION
Detected Pattern: ACCELERATION


## Contribution Functions

In [8]:
# 1. Moving Scenario on green Tl State
contribution_scenario1 = fetch_contribution(dist=1.0, velocity=10.0, acceleration=0.0, tl_state=TL_STATUS.GREEN.value, time_in_tl_state=5.0, is_lead=False, turn_type=TURN.RIGHT.value)
print(f"Contribution of scenario 1: {contribution_scenario1}")

# 2. Moving Scenario on red Tl State
contribution_scenario2 = fetch_contribution(dist=1.0, velocity=10.0, acceleration=0.0, tl_state=TL_STATUS.RED.value, time_in_tl_state=3,  is_lead=False, turn_type=TURN.RIGHT.value)
print(f"Contribution of scenario 2: {contribution_scenario2}")

# 3. Agent Drives through Intersection Shortly after Red Change
contribution_scenario2 = fetch_contribution(dist=1.0, velocity=10.0, acceleration=0.0, tl_state=TL_STATUS.RED.value, time_in_tl_state=0.1,  is_lead=False, turn_type=TURN.RIGHT.value)
print(f"Contribution of scenario 3: {contribution_scenario2}")


Contribution of scenario 1: 3
Contribution of scenario 2: -3
Contribution of scenario 3: -1


## Experimental Pattern-based Contribution 

In [9]:
scene_idx = 39
features = junctionDataset.get_combined_features( get_pickled_dataset_index(scene_idx, cfg)).query("scene_idx == @scene_idx") 
features["turn_type"] = features["lane_id"].apply(mapAPI.get_turn_type)

# calculation contribution
frame_contribution = apply_frame_contribution(features)

scene_prediction = generalize_over_frames(frame_contribution)
prediction = generalize_over_scenes_by_class_majority(scene_prediction)

# Append intersection information
prediction = prediction.rename(columns={"sequence_head_lane_id": "lane_id"})
prediction = prediction.merge(junctionDataset.get_branch_dataset(DATASET_TYPE.MINIMAL), on ="lane_id", how="left")
prediction

,lane_id,tl_id,all_assignments_counts,pos_assignment_counts,predicted_relation,junction_id,branch_id
0,/E65,4Xis,1,1,1,3Ccg,WX0Uk
1,/E65,CqSo,1,1,1,3Ccg,WX0Uk
2,/E65,DDUl,1,1,1,3Ccg,WX0Uk
3,/E65,ciok,1,0,0,3Ccg,WX0Uk
4,/E65,nuVl,1,1,1,3Ccg,WX0Uk
5,BE65,4Xis,1,1,1,3Ccg,WX0Uk
6,BE65,CqSo,1,1,1,3Ccg,WX0Uk
7,BE65,DDUl,1,1,1,3Ccg,WX0Uk
8,BE65,ciok,1,0,0,3Ccg,WX0Uk
9,BE65,nuVl,1,1,1,3Ccg,WX0Uk


## Example for one scene

In [10]:
heristic_prediction_for_signle_scene(scene_idx=2434, cfg= cfg, junctionDataset=junctionDataset, mapAPI=mapAPI)

,lane_id,tl_id,scene_relation,avg_scene_relation,predicted_relation,junction_id,branch_id
0,+xmM,/ggb,1,1.0,1,y4Ss,Evy5c
1,+xmM,LL1i,1,1.0,1,y4Ss,Evy5c
2,+xmM,NTTe,1,1.0,1,y4Ss,Evy5c
3,AxmM,/ggb,1,1.0,1,y4Ss,Evy5c
4,AxmM,LL1i,1,1.0,1,y4Ss,Evy5c
5,AxmM,NTTe,1,1.0,1,y4Ss,Evy5c
6,fxmM,/ggb,1,1.0,1,y4Ss,Evy5c
7,fxmM,LL1i,1,1.0,1,y4Ss,Evy5c
8,fxmM,NTTe,1,1.0,1,y4Ss,Evy5c


In [11]:
prediction_single_scene = heristic_prediction_for_signle_scene(scene_idx=39, cfg= cfg,  junctionDataset=junctionDataset, mapAPI=mapAPI)
prediction_single_scene

,lane_id,tl_id,scene_relation,avg_scene_relation,predicted_relation,junction_id,branch_id
0,/E65,4Xis,1,1.0,1,3Ccg,WX0Uk
1,/E65,CqSo,1,1.0,1,3Ccg,WX0Uk
2,/E65,DDUl,1,1.0,1,3Ccg,WX0Uk
3,/E65,ciok,0,1.0,0,3Ccg,WX0Uk
4,/E65,nuVl,1,1.0,1,3Ccg,WX0Uk
5,BE65,4Xis,1,1.0,1,3Ccg,WX0Uk
6,BE65,CqSo,1,1.0,1,3Ccg,WX0Uk
7,BE65,DDUl,1,1.0,1,3Ccg,WX0Uk
8,BE65,ciok,0,1.0,0,3Ccg,WX0Uk
9,BE65,nuVl,1,1.0,1,3Ccg,WX0Uk


In [12]:
branch_id = "WX0Uk"# Get the branch id from the static map and append direction (N/E/S/W)
number_of_scenes = 3

scenes_to_analyze = junctionDataset.get_scene_indices_for_branch(branch_id=branch_id)[:number_of_scenes] 
print("Scenes to analyse for branch:", scenes_to_analyze)

# Filter for specific branch because scenes also show other intersection!
prediction_multiple_scenes = heristic_prediction_for_scenes(scene_indices=scenes_to_analyze, cfg=cfg,  junctionDataset=junctionDataset, mapAPI=mapAPI).query("branch_id == @branch_id").reset_index(drop=True)
prediction_multiple_scenes

Scenes to analyse for branch: [38, 39, 74]


,lane_id,tl_id,scene_relation,avg_scene_relation,predicted_relation,junction_id,branch_id
0,/E65,4Xis,1,1.0,1,3Ccg,WX0Uk
1,/E65,CqSo,1,1.0,1,3Ccg,WX0Uk
2,/E65,DDUl,1,1.0,1,3Ccg,WX0Uk
3,/E65,ciok,0,1.0,0,3Ccg,WX0Uk
4,/E65,nuVl,1,1.0,1,3Ccg,WX0Uk
5,BE65,4Xis,3,3.0,1,3Ccg,WX0Uk
6,BE65,CqSo,3,3.0,1,3Ccg,WX0Uk
7,BE65,DDUl,3,3.0,1,3Ccg,WX0Uk
8,BE65,ciok,0,3.0,0,3Ccg,WX0Uk
9,BE65,nuVl,3,3.0,1,3Ccg,WX0Uk


In [13]:
columns = ["lane_id", "tl_id", "predicted_relation"]
prediction_single_scene[columns].equals(prediction_multiple_scenes[columns])

True

In [14]:
columns = ["lane_id", "tl_id", "predicted_relation"]
prediction[columns].equals(prediction_multiple_scenes[columns])

True